# Fine-Tuning NER Model for Amharic Text

## Overview
This notebook demonstrates how to fine-tune a Named Entity Recognition (NER) model specifically for Amharic text using the XLM-RoBERTa transformer model. The project focuses on identifying entities like products, prices, and locations in Amharic commercial text.

## Key Features
- Data preprocessing for CoNLL format
- Token alignment for transformer models
- Fine-tuning XLM-RoBERTa for multilingual NER
- Comprehensive evaluation metrics
- Model persistence and inference capabilities

## 1. Environment Setup and Library Installation

Installing all required dependencies for the NER fine-tuning process.

In [ ]:
# Install Required Libraries
!pip install transformers datasets torch accelerate evaluate seqeval
!pip install huggingface_hub --upgrade

## 2. Import Dependencies

Importing all necessary libraries for data processing, model training, and evaluation.

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import json
from datetime import datetime
import re
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import drive, files
import io

## 3. Google Drive Integration

Mounting Google Drive to access training data and save model artifacts.

In [ ]:
# Mount Google Drive to access your data
drive.mount('/content/drive')

## 4. Data Loading and Preprocessing Functions

Defining utility functions to load CoNLL format data and convert it into a structured format suitable for transformer models.

In [ ]:
def load_conll_data(file_path):
    """Load CoNLL format data and convert to structured format"""
    sentences = []
    labels = []
    
    current_sentence = []
    current_labels = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line == "":  # Empty line indicates sentence boundary
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                    current_sentence = []
                    current_labels = []
            else:
                parts = line.split('\t')
                if len(parts) >= 2:
                    token = parts[0]
                    label = parts[1]
                    current_sentence.append(token)
                    current_labels.append(label)
    
    # Add the last sentence if it doesn't end with empty line
    if current_sentence:
        sentences.append(current_sentence)
        labels.append(current_labels)
    
    return sentences, labels

## 5. Sample Data Generation

Creating sample CoNLL format data for demonstration purposes with Amharic text and entity labels.

In [ ]:
def create_sample_conll_data():
    """Create sample CoNLL data for demonstration"""
    sample_data = """LIFESTAR	B-PRODUCT
1	I-PRODUCT
Million	I-PRODUCT
4K	I-PRODUCT
Android	I-PRODUCT
ሪሲቨር	I-PRODUCT
ዋጋ	B-PRICE
7000	I-PRICE
ብር	I-PRICE
ነው	O
.	O

MAGIC	B-PRODUCT
REMOTE	I-PRODUCT
በ	O
ማራኪ	B-LOC
ሳት	I-LOC
አለ	O
.	O

የመኪና	B-PRODUCT
ANDROID	I-PRODUCT
SCREEN	I-PRODUCT
በጥራት	O
ዋጋ	B-PRICE
በግማሽ	I-PRICE
የቀነሰ	I-PRICE
.	O

Xcruiser	B-PRODUCT
Magic	I-PRODUCT
Box	I-PRODUCT
መርካቶ	B-LOC
አንዋር	I-LOC
መስጂድ	I-LOC
ጎንደር	I-LOC
ውስጥ	O
.	O"""
    
    # Save sample data to file
    with open('/content/sample_conll_data.txt', 'w', encoding='utf-8') as f:
        f.write(sample_data)
    
    return '/content/sample_conll_data.txt'

## 6. Data Upload and Loading

Uploading the CoNLL format training data and loading it into memory for processing.

In [ ]:
# Upload your CoNLL data file
print("Please upload your CoNLL format labeled data file:")
uploaded = files.upload()
conll_file_path = list(uploaded.keys())[0] if uploaded else create_sample_conll_data()

# Load the data
sentences, labels = load_conll_data(conll_file_path)
print(f"Loaded {len(sentences)} sentences")
print(f"Sample sentence: {sentences[0]}")
print(f"Sample labels: {labels[0]}")

## 7. Label Mapping Creation

Creating bidirectional mappings between entity labels and their corresponding IDs for model training.

In [ ]:
# Create label mappings
unique_labels = set()
for label_seq in labels:
    unique_labels.update(label_seq)

label_list = sorted(list(unique_labels))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

print(f"Unique labels: {label_list}")
print(f"Number of labels: {len(label_list)}")

## 8. Tokenization and Label Alignment

Implementing the crucial tokenization function that aligns entity labels with subword tokens generated by the transformer tokenizer.

In [ ]:
def tokenize_and_align_labels(examples, tokenizer, label_to_id):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        padding=False
    )
    
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## 9. Dataset Preparation and Splitting

Preparing the dataset by splitting it into training, validation, and test sets for robust model evaluation.

In [ ]:
# Prepare Dataset
df = pd.DataFrame({
    'tokens': sentences,
    'labels': labels
})

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

## 10. Model and Tokenizer Initialization

Loading the pre-trained XLM-RoBERTa model and tokenizer, configured for token classification with our specific entity labels.

In [ ]:
# Model Setup
MODEL_NAME = "xlm-roberta-base"  # You can also try "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

## 11. Dataset Tokenization

Applying tokenization and label alignment to all dataset splits for model training compatibility.

In [ ]:
# Tokenize datasets
train_tokenized = train_dataset.map(
    lambda x: tokenize_and_align_labels(x, tokenizer, label_to_id),
    batched=True
)
val_tokenized = val_dataset.map(
    lambda x: tokenize_and_align_labels(x, tokenizer, label_to_id),
    batched=True
)
test_tokenized = test_dataset.map(
    lambda x: tokenize_and_align_labels(x, tokenizer, label_to_id),
    batched=True
)

## 12. Data Collator Setup

Configuring the data collator for dynamic padding and batch preparation during training.

In [ ]:
# Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, 
    padding=True
)

## 13. Evaluation Metrics Configuration

Defining comprehensive evaluation metrics including precision, recall, F1-score, and accuracy using seqeval for proper sequence evaluation.

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    results = {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "accuracy": accuracy_score(true_labels, true_predictions),
    }
    return results

## 14. Training Configuration

Setting up comprehensive training arguments including learning rate, batch sizes, evaluation strategy, and model saving parameters.

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    report_to=None,  # Disable wandb logging
    dataloader_pin_memory=False,
)

## 15. Trainer Initialization

Initializing the Hugging Face Trainer with our model, datasets, and configuration parameters.

In [ ]:
# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

## 16. Model Training

Starting the fine-tuning process. This cell will train the model for the specified number of epochs with automatic evaluation and checkpointing.

In [ ]:
# Train the Model
print("Starting training...")
trainer.train()

## 17. Model Evaluation

Evaluating the trained model on the held-out test set to assess final performance metrics.

In [ ]:
# Evaluate on Test Set
test_results = trainer.evaluate(test_tokenized)
print(f"Test Results: {test_results}")

## 18. Model Persistence

Saving the fine-tuned model and tokenizer for future use and deployment.

In [ ]:
# Save the Model
model.save_pretrained("./xlm-roberta-amharic-ner")
tokenizer.save_pretrained("./xlm-roberta-amharic-ner")

print("Model saved successfully!")

## 19. Inference Function Implementation

Implementing a utility function for making predictions on new text using the trained model.

In [ ]:
def predict_entities(text, model, tokenizer, id_to_label):
    tokens = text.split()
    inputs = tokenizer(tokens, truncation=True, is_split_into_words=True, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    predictions = torch.argmax(outputs.logits, dim=2)
    
    word_ids = inputs.word_ids()
    previous_word_idx = None
    predicted_labels = []
    
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != previous_word_idx:
            predicted_labels.append(id_to_label[predictions[0][i].item()])
        previous_word_idx = word_idx
    
    return list(zip(tokens, predicted_labels))

## 20. Model Testing and Demonstration

Testing the trained model with sample Amharic text to demonstrate entity recognition capabilities.

In [ ]:
# Test the Model
test_text = "LIFESTAR 1 Million 4K Android ሪሲቨር ዋጋ 7000 ብር መርካቶ ውስጥ አለ"
predictions = predict_entities(test_text, model, tokenizer, id_to_label)
print(f"Test text: {test_text}")
print(f"Predictions: {predictions}")

## Conclusion

This notebook successfully demonstrates the complete pipeline for fine-tuning a multilingual transformer model for Amharic NER tasks. The model can now identify:

- **Products**: Electronic devices and commercial items
- **Prices**: Monetary values and currency information  
- **Locations**: Geographic places and addresses

The fine-tuned model is ready for deployment and can be further improved with additional training data or hyperparameter optimization.